In [ ]:
import requests

BASE_URL = "http://localhost:8000"

# Start a session so cookies persist between requests
session = requests.Session()

# 1. Fetch CSRF token (and laravel_session cookie)
resp = session.get(f"{BASE_URL}/csrf-token")
print("CSRF token response:", resp.json())
csrf_token = resp.json()["token"]

# 2. Perform login using that token
login_payload = {
    "email": "test@example.com",
    "password": "password123"
}

headers = {
    "X-CSRF-TOKEN": csrf_token,
    "Accept": "application/json"
}

resp = session.post(f"{BASE_URL}/login", json=login_payload, headers=headers)
print("Login status code:", resp.status_code)
print("Login response JSON:", resp.json())


CSRF token response: {'token': 'osjTGSUCvoDY216GUvfudiOsJKTH7Kr01hECMfe3'}
Login status code: 422
Login response JSON: {'message': 'The provided credentials are incorrect.', 'errors': {'email': ['The provided credentials are incorrect.']}}


## Test rate limiting

In [ ]:
import requests
import time

# Base URL of your Laravel app
base_url = "http://localhost:8000"

# Get CSRF token first (since you're using web middleware)
session = requests.Session()

# Get CSRF token
csrf_response = session.get(f"{base_url}/csrf-token")
csrf_token = csrf_response.json()['token']

print(f"CSRF Token: {csrf_token}")

# Test data
login_data = {
    'email': 'test@example.com',
    'password': 'wrongpassword'
}

# Headers
headers = {
    'X-CSRF-TOKEN': csrf_token,
    'X-Requested-With': 'XMLHttpRequest',
    'Content-Type': 'application/json'
}

print("\nTesting rate limiting...")
print("=" * 40)

# Try 7 login attempts
for i in range(1, 8):
    print(f"Attempt {i}: ", end="")
    
    try:
        response = session.post(
            f"{base_url}/login",
            json=login_data,
            headers=headers
        )
        
        print(f"Status: {response.status_code}", end="")
        
        if response.status_code == 429:
            print(" - RATE LIMITED! ✅")
            print(f"Response: {response.text}")
            break
        elif response.status_code == 422:
            print(" - Validation error (expected)")
        elif response.status_code == 419:
            print(" - CSRF token expired")
            break
        else:
            print(f" - {response.reason}")
            
    except Exception as e:
        print(f"Error: {e}")
    
    time.sleep(0.5)  # Small delay between requests

print("\nTest completed!")

CSRF Token: tf2on4BZJZ3Wyy8XCz3LJvC2fuFAkAav74zIGki7

Testing rate limiting...
Attempt 1: Status: 422 - Validation error (expected)
Attempt 2: Status: 422 - Validation error (expected)
Attempt 3: Status: 422 - Validation error (expected)
Attempt 4: Status: 422 - Validation error (expected)
Attempt 5: Status: 422 - Validation error (expected)
Attempt 6: Status: 429 - RATE LIMITED! ✅
Response: {
    "message": "Too Many Attempts.",
    "exception": "Illuminate\\Http\\Exceptions\\ThrottleRequestsException",
    "file": "C:\\Users\\Jandrik\\Documents\\MyFILES\\TUA ITSE 4th Year\\IAS 2\\sample-app-ITPC174\\backend\\vendor\\laravel\\framework\\src\\Illuminate\\Routing\\Middleware\\ThrottleRequests.php",
    "line": 244,
    "trace": [
        {
            "file": "C:\\Users\\Jandrik\\Documents\\MyFILES\\TUA ITSE 4th Year\\IAS 2\\sample-app-ITPC174\\backend\\vendor\\laravel\\framework\\src\\Illuminate\\Routing\\Middleware\\ThrottleRequests.php",
            "line": 154,
            "function

In [1]:
# Looking to send emails in production? Check out our Email API/SMTP product!
import smtplib

sender = "Private Person <from@example.com>"
receiver = "A Test User <linkanjarad@yahoo.com>"

message = f"""\
Subject: Hi Mailtrap
To: {receiver}
From: {sender}

This is a test e-mail message."""

with smtplib.SMTP("sandbox.smtp.mailtrap.io", 2525) as server:
    server.starttls()
    server.login("7d85fd292158ae", "a0c148cff0654b")
    server.sendmail(sender, receiver, message)

In [ ]:
import requests
import json

# Request password reset
response = requests.post('http://localhost:8000/forgot-password', 
                        json={'email': 'test@example.com'},
                        headers={'X-Requested-With': 'XMLHttpRequest'})

print("Reset request response:", response.json())
print("Status code:", response.status_code)

Reset request response: {'message': 'CSRF token mismatch.', 'exception': 'Symfony\\Component\\HttpKernel\\Exception\\HttpException', 'file': 'C:\\Users\\Jandrik\\Documents\\MyFILES\\TUA ITSE 4th Year\\IAS 2\\sample-app-ITPC174\\backend\\vendor\\laravel\\framework\\src\\Illuminate\\Foundation\\Exceptions\\Handler.php', 'line': 674, 'trace': [{'file': 'C:\\Users\\Jandrik\\Documents\\MyFILES\\TUA ITSE 4th Year\\IAS 2\\sample-app-ITPC174\\backend\\vendor\\laravel\\framework\\src\\Illuminate\\Foundation\\Exceptions\\Handler.php', 'line': 617, 'function': 'prepareException', 'class': 'Illuminate\\Foundation\\Exceptions\\Handler', 'type': '->'}, {'file': 'C:\\Users\\Jandrik\\Documents\\MyFILES\\TUA ITSE 4th Year\\IAS 2\\sample-app-ITPC174\\backend\\vendor\\laravel\\framework\\src\\Illuminate\\Routing\\Pipeline.php', 'line': 51, 'function': 'render', 'class': 'Illuminate\\Foundation\\Exceptions\\Handler', 'type': '->'}, {'file': 'C:\\Users\\Jandrik\\Documents\\MyFILES\\TUA ITSE 4th Year\\IAS 2

In [ ]:
import requests
from urllib.parse import unquote

# Create a session to persist cookies
session = requests.Session()

# Base URL - adjust to your Laravel app
base_url = 'http://localhost:8000'

print("=== CSRF Token Debugging ===\n")

# Step 1: Get CSRF cookie
print("1. Getting CSRF cookie...")
response = session.get(f'{base_url}/sanctum/csrf-cookie')
print(f"   Status: {response.status_code}")
print(f"   Cookies: {session.cookies}")

# Step 2: Extract CSRF token from cookies
csrf_token = None
for cookie in session.cookies:
    if cookie.name == 'XSRF-TOKEN':
        csrf_token = unquote(cookie.value)  # Decode URL encoding
        print(f"   XSRF-TOKEN (decoded): {csrf_token}")
    elif cookie.name == 'laravel_session':
        print(f"   laravel_session: {cookie.value[:50]}...")

# Step 3: Try login with the token
if csrf_token:
    print("\n2. Attempting login with CSRF token...")
    login_data = {
        'email': 'linkanjarad@yahoo.com',  # Replace with valid test user
        'password': 'password'      # Replace with valid password
    }
    
    headers = {
        'X-CSRF-TOKEN': csrf_token,
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    
    login_response = session.post(
        f'{base_url}/api/login',
        json=login_data,
        headers=headers
    )
    
    print(f"   Login Status: {login_response.status_code}")
    print(f"   Login Response: {login_response.text}")
    
    if login_response.status_code == 200:
        print("   ✅ Login successful!")
    else:
        print("   ❌ Login failed")
        
        # Check if it's a CSRF token issue
        if "CSRF token mismatch" in login_response.text:
            print("   🔍 CSRF token mismatch detected!")
        elif login_response.status_code == 419:
            print("   🔍 419 error - CSRF token mismatch or expired")

# Step 4: Inspect all cookies in detail
print("\n3. Detailed Cookie Information:")
for cookie in session.cookies:
    print(f"   Name: {cookie.name}")
    print(f"   Value: {cookie.value}")
    print(f"   Domain: {cookie.domain}")
    print(f"   Path: {cookie.path}")
    print(f"   Secure: {cookie.secure}")
    print(f"   HttpOnly: {cookie.has_nonstandard_attr('HttpOnly')}")
    print("   ---")

# Step 5: Manual token comparison (if you want to verify)
print("\n4. Manual Verification:")
print("   Open your browser DevTools:")
print("   1. Go to Application → Cookies")
print("   2. Find XSRF-TOKEN and laravel_session")
print("   3. Compare with the values above")

=== CSRF Token Debugging ===

1. Getting CSRF cookie...
   Status: 204
   Cookies: <RequestsCookieJar[<Cookie XSRF-TOKEN=eyJpdiI6IkJNNHdqOE9mSkVpMjB4cERSZlRhT2c9PSIsInZhbHVlIjoiSkRxOSt0VVFhRVF3M0Uxald0MTlnY1MvOEJPV0JaVkczR3RqeCszSEE4bTIwS0pTaHJkbldaeURBVmJMT2xkTUFTY0dHMXl2RlJHZjhNeUQ0OTREVDBUUENXeFpCNk1BTWRZVWUxWkVlU3ZaaHpSdE1YUEt3OU1NSkhXTDFXNW0iLCJtYWMiOiI2ODYxNjZiZTljNmQxNzZhYmYxZjUxYmQyNDFmZjdlZWNhOTRkZjA4MmU5OWNiYjNmMDFjOGQ0MWE4MGNkZDFkIiwidGFnIjoiIn0%3D for localhost.local/>, <Cookie laravelreactapp_session=eyJpdiI6InkyYlRGSktpTzZkdVAvM1c1bDE0ZUE9PSIsInZhbHVlIjoieWE0Q3NJRW9Eb2Fka21pR0RQeDBTcUpDa2pKd2hhbXBuVGdBallhZkpjNkN5SysrTlk0ZDdCTnArVmZQMWJCSm9yMThzYVoxTlpIdVBVNGgvb2U2WlVYdE1WbjloRnRaTkd2azZabzJFdS9EM2JVRXEzVmZFd0NjL1cyMmZUNkYiLCJtYWMiOiJlYTFkN2U5ZWQ5NGUyNGVmNzRmNGRhMTVkZjk4NGFmNWM3ZDRhYmU2MDU1Yzg3ZTA1ZjRmOGJhYTE3MWE0OWMzIiwidGFnIjoiIn0%3D for localhost.local/>, <Cookie dsrRzmIDVzlAALn7NOsPeu3qHFKJU50IQGVblFby=eyJpdiI6IkgrR0hDYmJNTU1wSGl1SUVWYUFTRXc9PSIsInZhbHVlIjoiZ3lzdStPSk

In [2]:
import os

def print_tree(root_path, prefix="", exclude=None):
    """
    Recursively print a formatted directory tree.

    Parameters:
        root_path (str): The directory to print.
        prefix (str): Internal use for formatting.
        exclude (list[str]): Folder names to skip, printing ellipsis instead.
    """
    if exclude is None:
        exclude = []

    # Get the list of entries in the current directory
    entries = sorted(os.listdir(root_path))
    entries_count = len(entries)
    
    for i, entry in enumerate(entries):
        path = os.path.join(root_path, entry)
        connector = "└── " if i == entries_count - 1 else "├── "
        print(prefix + connector + entry)

        if os.path.isdir(path):
            if entry in exclude:
                # Show an ellipsis and don't recurse
                extension = "    " if i == entries_count - 1 else "│   "
                print(prefix + extension + "...")
            else:
                extension = "    " if i == entries_count - 1 else "│   "
                print_tree(path, prefix + extension, exclude=exclude)


# Example usage:
print_tree("../sample-app-itpc174", exclude=["node_modules", ".git", "vendor"])


├── .git
│   ...
├── .gitignore
├── backend
│   ├── .editorconfig
│   ├── .env
│   ├── .env.example
│   ├── .styleci.yml
│   ├── CHANGELOG.md
│   ├── Dockerfile
│   ├── README.md
│   ├── app
│   │   ├── Console
│   │   │   └── Commands
│   │   │       └── DeleteUser.php
│   │   ├── Http
│   │   │   └── Controllers
│   │   │       ├── Auth
│   │   │       │   └── VerifyEmailController.php
│   │   │       ├── AuthController.php
│   │   │       └── Controller.php
│   │   ├── Models
│   │   │   └── User.php
│   │   ├── Notifications
│   │   │   └── ResetPasswordNotification.php
│   │   └── Providers
│   │       └── AppServiceProvider.php
│   ├── artisan
│   ├── bootstrap
│   │   ├── app.php
│   │   ├── cache
│   │   │   ├── .gitignore
│   │   │   ├── packages.php
│   │   │   └── services.php
│   │   └── providers.php
│   ├── composer.json
│   ├── composer.lock
│   ├── config
│   │   ├── app.php
│   │   ├── auth.php
│   │   ├── cache.php
│   │   ├── cors.php
│   │   ├── database.php
│   │  